pip install torch transformers sklearn pandas

For Mac M1:

curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm Elon Musk,", max_length=30, num_return_sequences=5)

The next step is to use all tweets to build a TextDataset. The TextDataset is a custom implementation of the Pytroch Dataset class implemented by the transformers library. 

First, we split the tweets into a train and test section then write them into a train_dataset.txt and test_dataset.txt



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter

data = pd.read_csv("dataset/train_cleaned.csv")['content'].to_numpy()
train, test = train_test_split(data,test_size=0.15)
traindata = ''
testdata = ''
for i in train:
    traindata += i.replace("&amp", "") +'\n'
f = open('train_dataset.txt','w')
f.write(traindata)
for i in test:
    testdata += i.replace("&amp","") +'\n'
f = open('test_dataset.txt','w')
f.write(testdata)

The next step is to download the tokenizer. We use the tokenizer from the german-gpt2 model.

In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
#if tokenizer.pad_token is None:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

/opt/anaconda3/envs/DL/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
with open(train_path, encoding="utf-8") as f:
    data = f.readlines()
data = [line.strip() for line in data if len(line) > 0 and not line.isspace()]
batch_encoding = tokenizer(data, add_special_tokens=False, padding=False, truncation=True, max_length=128)

In [ ]:
import torch 
print(batch_encoding['input_ids'])
examples = batch_encoding["input_ids"]
examples = [{"input_ids": torch.tensor(e, dtype=torch.long)} for e in examples]

In [ ]:
for i in batch_encoding['input_ids']:
    print(tokenizer.convert_ids_to_tokens(i))

In [2]:
from transformers import TextDataset,DataCollatorForLanguageModeling,LineByLineTextDataset

def load_dataset(train_path,test_path,tokenizer):
    train_dataset_LineByLine = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset_LineByLine = LineByLineTextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False, mlm_probability=0.15
    )
    return train_dataset_LineByLine, test_dataset_LineByLine, train_dataset,test_dataset,data_collator

train_dataset_LineByLine, test_dataset_LineByLine, train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)


/opt/anaconda3/envs/DL/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/opt/anaconda3/envs/DL/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
from transformers import LineByLineTextDataset
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./gpt2-musk", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=1, # number of training epochs
    per_device_train_batch_size=1, # batch size for training
    #per_device_eval_batch_size=32,  # batch size for evaluation
    #eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    report_to="tensorboard"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_LineByLine,
    #eval_dataset=test_dataset,
    #prediction_loss_only=True,
)

loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /Users/yu/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params"

In [ ]:
for id, tensor  in enumerate(train_dataset):
    print(type(tensor))
    print(tokenizer.convert_ids_to_tokens(tensor))
    break

In [ ]:
type(train_dataset_LineByLine)

In [ ]:
for id, tensor  in enumerate(train_dataset_LineByLine):
    print(type(tensor))
    print(tokenizer.convert_ids_to_tokens(tensor))
    break

In [8]:
trainer.train()

***** Running training *****
  Num examples = 14262
  Num Epochs = 1
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 14262













































































KeyboardInterrupt: 

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

tweet = pipeline('text-generation',model='gpt2-musk', tokenizer=tokenizer )

In [ ]:
#generator = pipeline('text-generation', model='gpt2')
from transformers import pipeline, set_seed
set_seed(42)
tweet("With steel membrane wings like a Dragon,", max_length=50, num_return_sequences=5)